In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Initialize pygame for sound playback
pygame.mixer.init()
pygame.mixer.music.load("C:/Users/kiran yadav/Downloads/mixkit-classic-alarm-995.wav")

# MediaPipe face mesh
fp = mp.solutions.face_mesh
fm_model = fp.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

# Constants
CLOSED_EYE_THRESH = 0.15 # minimum eye threshold value when it open is 0.20 but this video purpose iam taken this value
YAWN_THRESH = 0.6
YAWN_COOLDOWN = 4

yawn_count = 0
last_yawn_time = 0
eye_closed_start = None

# style for connections
simple_style = mp.solutions.drawing_utils.DrawingSpec(color=(193, 166, 144), thickness=1, circle_radius=1)

def get_landmark(frame, landmarks, idx):
    h, w, _ = frame.shape
    lm = landmarks[idx]
    return np.array([lm.x * w, lm.y * h])


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = fm_model.process(rgb)
    
    current_time = time.time()

    if results.multi_face_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(image=frame,
                                                 landmark_list = results.multi_face_landmarks[0],
                                                 connections = fp.FACEMESH_LEFT_EYE.union(fp.FACEMESH_RIGHT_EYE).union(fp.FACEMESH_LIPS),
                                                 landmark_drawing_spec=None,
                                                 connection_drawing_spec = simple_style)
        landmarks = results.multi_face_landmarks[0].landmark

        # EAR (Eye Aspect Ratio)
        def EAR(p1, p2, p3, p4, p5, p6):
            A = np.linalg.norm(get_landmark(frame, landmarks, p2) - get_landmark(frame, landmarks, p6))
            B = np.linalg.norm(get_landmark(frame, landmarks, p3) - get_landmark(frame, landmarks, p5))
            C = np.linalg.norm(get_landmark(frame, landmarks, p1) - get_landmark(frame, landmarks, p4))
            return (A + B) / (2.0 * C)

        left_eye = EAR(362, 385, 387, 263, 373, 380)
        right_eye = EAR(33, 160, 158, 133, 153, 144)
        avg_ear = (left_eye + right_eye) / 2.0

        # MAR (Mouth Aspect Ratio) for yawn
        A = np.linalg.norm(get_landmark(frame, landmarks, 81) - get_landmark(frame, landmarks, 178))
        B = np.linalg.norm(get_landmark(frame, landmarks, 13) - get_landmark(frame, landmarks, 14))
        C = np.linalg.norm(get_landmark(frame, landmarks, 311) - get_landmark(frame, landmarks, 308))
        D = np.linalg.norm(get_landmark(frame, landmarks, 61) - get_landmark(frame, landmarks, 291))
        mar = (A + B + C) / (2.0 * D)

        # Yawn Detection
        if mar > YAWN_THRESH and (current_time - last_yawn_time > YAWN_COOLDOWN):
            yawn_count += 1
            last_yawn_time = current_time


        # Alarm for long eye closure
        if avg_ear < CLOSED_EYE_THRESH:
            if eye_closed_start is None:
                eye_closed_start = current_time
            elif current_time - eye_closed_start >= 2:
                if not pygame.mixer.music.get_busy():
                    pygame.mixer.music.play()
                cv2.putText(frame, "Wake Up!", (30, 180),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        else:
            eye_closed_start = None
            if pygame.mixer.music.get_busy():
                pygame.mixer.music.stop()
            cv2.putText(frame, "AWAKE", (30, 180),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

        # Draw values
        cv2.putText(frame, f'EAR: {avg_ear:.2f}', (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        cv2.putText(frame, f'MAR: {mar:.2f}', (30, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
        cv2.putText(frame, f'Yawns: {yawn_count}', (30, 140),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)

    # Show the frame
    cv2.imshow("Drowsiness Detection", frame)

    # Exit
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
pygame.mixer.music.stop()
